<a href="https://colab.research.google.com/github/andrea212023/ML-Kis/blob/main/HW_Prompt_Langchain_KIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Завдання 1: Виклик LLM з базовим промптом

Створіть можливість викликати LLM зі звичайним текстовим промптом.

Промпт має дозвляти отримати інформацію простою мовою на певну тему. В цьому завданні ми хочемо дізнатись про тему "Квантові обчислення".

Відповідь моделі повинна містити визначення, ключові переваги та поточні дослідження в цій галузі.

Обмежте відповідь до 200 символів і пропишіть в промпті аби відповідь була короткою (це зекономить Вам час і гроші на згенеровані токени).

В якості LLM можна скористатись як моделлю з HugginFace (рекомендую Mistral), так і ChatGPT4 або ChatGPT3. В обох випадках треба імпортувати потрібну "обгортку" (тобто клас, який дозволить ініціювати модель) з LangChain для виклику LLM за API, а також зчитати особистий токен з файла, наприклад, `creds.json`, який розміщений у Вас локально і Ви НЕ здаєте його в ДЗ і НЕ комітите в git 😏

Встановіть своє значення температури на свій розсуд (тут немає правильного чи неправильного значення) і напишіть, чому ви обрали саме таке значення для цього завдання.  

Запити можна робити як українською, так і англійською - орієнтуйтесь на те, де і чи хочете ви потім лишити цей проєкт і відповідна яка мова буде пасувати більше. В розвʼязках промпти - українською.

In [ ]:
!pip install --upgrade langchain

In [ ]:
!pip -q install langchain langchain_community huggingface_hub openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [ ]:
creds = '/content/drive/MyDrive/Colab Notebooks/data/creds.json'

# Read creds.json
with open(creds, 'r') as f:
    creds = json.load(f)

api_key = creds["openai_token"]


In [ ]:
import time

# Initialize the model using the API key with a temperature of 0.3 for less randomness
llm = OpenAI(api_key=api_key, temperature=0.3)

# Example prompt asking for a brief answer on the topic of quantum computing
prompt = """
Квантові обчислення: визначення, ключові переваги та поточні дослідження в галузі.
Коротко і до 200 символів, будь ласка.
"""

# Implement retry mechanism with exponential backoff
max_retries = 5
retry_delay = 1  # Initial delay in seconds

for attempt in range(max_retries):
    try:
        # Get the model's response
        response = llm(prompt)
        # Print the response
        print(response)
        break  # Exit the loop if successful
    except Exception as e:
        if "429" in str(e):
            print(f"Rate limit exceeded (attempt {attempt + 1}/{max_retries}). Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            retry_delay *= 2  # Exponential backoff
        else:
            print(f"An error occurred: {e}")
            break  # Exit the loop for other errors
else:
    print("Failed to get a response after multiple retries.")

Квантові обчислення - це обчислювальна технологія, яка використовує квантові біти замість класичних бітів для зберігання та обробки інформації. Вони мають потенціал вирішувати складні задачі швидше, ефективніше та точніше, ніж традиційні комп'ютери. Ключові переваги квантових обчислень - це висока швидкість та потужність обчислень, здатність до розв'язання складних задач, які недосяжні для класичних комп'ютерів, та можливість використання в галузях


**Explanation for temperature:**

I choose a temperature setting of 0.3 because it strikes a balance between creativity and accuracy. For tasks requiring concise and informative responses that stay on topic, a low temperature setting (like 0.3) helps minimize random variations in the answers, while maintaining clarity and structure.

### Завдання 2: Створення параметризованого промпта для генерації тексту
Тепер ми хочемо оновити попередній фукнціонал так, аби в промпт ми могли передавати тему як параметр. Для цього скористайтесь `PromptTemplate` з `langchain` і реалізуйте параметризований промпт та виклик моделі з ним.

Запустіть оновлений функціонал (промпт + модел) для пояснень про теми
- "Баєсівські методи в машинному навчанні"
- "Трансформери в машинному навчанні"
- "Explainable AI"

Виведіть результати відпрацювання моделі на екран.

In [ ]:
# Creating a Prompt Template
template = """
Поясніть тему "{topic}" простими словами. Будь ласка, дайте коротке пояснення.
"""

# Create a PromptTemplate object with the "topic" parameter
prompt_template = PromptTemplate(
    input_variables=["topic"],
    template=template,
)

# Topics to explain
topics = [
   "Баєсівські методи в машинному навчанні",
"Трансформери в машинному навчанні",
"Explainable AI",
]

# Loop to generate explanations for each topic
for topic in topics:
    # Generate prompt using parameter
    prompt = prompt_template.format(topic=topic)

    # Getting response from model
    response = llm(prompt)

    # Output the result
    print(f"Topic: {topic}\nAnswer: {response}\n")

Topic: Баєсівські методи в машинному навчанні
Answer: 
Баєсівські методи в машинному навчанні - це підхід до розв'язання задач штучного інтелекту, який базується на теорії ймовірності і використовує статистичні методи для прийняття рішень. Вони дозволяють моделювати невизначеність та непевність в даних та враховувати нову інформацію для поліпшення результатів. Ці методи широко використовуються в різних областях, наприклад, для розпізнавання образів, класифікації даних, прогнозування та рекомендацій. 

Topic: Трансформери в машинному навчанні
Answer: 
Трансформери в машинному навчанні - це алгоритми, які використовуються для обробки текстової інформації. Вони використовуються для перетворення слів у числа, що дозволяє комп'ютеру зрозуміти текст і виконувати завдання, пов'язані з мовою, наприклад, переклад або розпізнавання мови. Трансформери допомагають комп'ютеру "вчитися" мові, аналізуючи великі обсяги тексту і виявляючи закономірності в ньому. Вони є важливим інструментом для розвитк



### Завдання 3: Використання агента для автоматизації процесів
Створіть агента, який допоможе автоматично шукати інформацію про останні наукові публікації в різних галузях. Наприклад, агент має знайти 5 останніх публікацій на тему штучного інтелекту.

**Кроки:**
1. Налаштуйте агента типу ReAct в LangChain для виконання автоматичних запитів.
2. Створіть промпт, який спрямовує агента шукати інформацію в інтернеті або в базах даних наукових публікацій.
3. Агент повинен видати список публікацій, кожна з яких містить назву, авторів і короткий опис.

Для взаємодії з пошуком там необхідно створити `Tool`. В лекції ми використовували `serpapi`. Можна продовжити користуватись ним, або обрати інше АРІ для пошуку (вони в тому числі є безкоштовні). Перелік різних АРІ, доступних в langchain, і орієнтир по вартості запитів можна знайти в окремому документі [тут](https://hannapylieva.notion.site/API-12994835849480a69b2adf2b8441cbb3?pvs=4).

Лишаю також нижче приклад використання одного з безкоштовних пошукових АРІ - DuckDuckGo (не потребує створення токена!)  - можливо він вам сподобається :)


In [ ]:
!pip install -q langchain_community duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Obama's first name?")

'The first president, ... Barack Obama (b. 1961) ... President-elect. List of presidents of the United States from 1789 - till date. No. Portrait Name (Birth-Death) Term Party Election Vice President-elect 47: Donald Trump (b. 1946) [77] To be sworn in on January 20, 2025 Republican: 2024: 2 of 2. Barack Obama: timeline Key events in the life of Barack Obama. Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005-08). Here is a list of the presidents and vice presidents of the United States along with their parties and dates in office. ... Chester A Arthur: Twenty-First President of the United States. 10 Interesting Facts About James Buchanan. Martin Van Buren - Eighth President of the United States. Quotes From Harry S. Truman. Michelle LaVaughn Robinson Obama [1] (née Robinson; born January 17, 1

In [ ]:
# Initialization requested by DuckDuckGo
search = DuckDuckGoSearchRun()

# Performing a search for publications on the topic of "artificial intelligence"
query = "latest groundbreaking scientific publications on artificial intelligence"
results = search.invoke(query)

# Output the result by lines
formatted_results = results.split('. ') # Break the text into sentences

# Display each sentence separately
for sentence in formatted_results:
    print(sentence.strip() + ".\n")

Distributional Graphormer (DIG) is a deep learning framework for predicting protein structures with greater accuracy, a fundamental problem in molecular science.

This advance could help deliver breakthroughs in critical research areas like materials science and drug discovery.

The proliferation of artificial intelligence tools in scientific research risks creating illusions of understanding, where&nbsp;scientists believe they understand more about the world than they ...

Unfortunately, other discussions, such as those on artificial general intelligence (AGI) and 'superintelligence', are based on an understanding of AI that is more rooted in science fiction ...

AI is rapidly advancing science, with breakthroughs in fields like protein structure prediction, brain mapping, and flood forecasting.

These advancements are built on collaborations between researchers, technologists, and policymakers, and they offer a blueprint for how AI can improve human life.

The rapid advances in artif

**Improvement of query:**

Let's look for our request 'artificial intelligence' in semantic scholar.

In [ ]:
import requests

# Function for performing a search for publications
def search_publications(query, limit=5):
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": query,
        "fields": "title,authors,year,url",
        "limit": limit
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json().get("data", [])
    else:
        print("Error:", response.status_code)
        return []

# Search publications on the topic "artificial intelligence"
query = "artificial intelligence"
publications = search_publications(query)

# Output results: Title, authors, year and link
for pub in publications:
    title = pub.get('title', 'No title')
    authors = ', '.join([author['name'] for author in pub.get('authors', [])])
    year = pub.get('year', 'No year')
    url = pub.get('url', 'No URL')
    print(f"Title: {title}\nAuthors: {authors}\nYear: {year}\nURL: {url}\n")

Title: Explainable Artificial Intelligence (XAI): Concepts, Taxonomies, Opportunities and Challenges toward Responsible AI
Authors: Alejandro Barredo Arrieta, Natalia Díaz Rodríguez, J. Ser, Adrien Bennetot, S. Tabik, A. Barbado, S. García, S. Gil-Lopez, D. Molina, Richard Benjamins, Raja Chatila, Francisco Herrera
Year: 2019
URL: https://www.semanticscholar.org/paper/530a059cb48477ad1e3d4f8f4b153274c8997332

Title: Sparks of Artificial General Intelligence: Early experiments with GPT-4
Authors: Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, J. Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Y. Lee, Yuan-Fang Li, Scott M. Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang
Year: 2023
URL: https://www.semanticscholar.org/paper/8dbd57469bb32e6d57f23f5e765bf1c9ac8e080c

Title: High-performance medicine: the convergence of human and artificial intelligence
Authors: E. Topol
Year: 2019
URL: https://www.semanticscholar.org/paper/f134abeaf9bfd41f29b97aec675ec31895bf541d

Tit

**Improvement of query:**

Articles output by semantic scholar is too old.
Let's look for our request 'artificial intelligence' in arXiv.

In [ ]:
import requests

def search_publications_arxiv(query, max_results=5):
    """
    Searches arXiv for publications based on a given query.

    Args:
        query: The search query string.
        max_results: The maximum number of results to return.

    Returns:
        A list of dictionaries, where each dictionary represents a publication
        with 'title', 'authors', 'published', and 'summary' keys.
    """
    base_url = "http://export.arxiv.org/api/query"
    params = {
        "search_query": query,
        "max_results": max_results,
        "sortBy": "lastUpdatedDate",  # Sort by last updated date
        "sortOrder": "descending"
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

    # Parse the XML response
    import xml.etree.ElementTree as ET
    root = ET.fromstring(response.content)
    entries = root.findall("{http://www.w3.org/2005/Atom}entry")

    publications = []
    for entry in entries:
        title = entry.find("{http://www.w3.org/2005/Atom}title").text
        authors = ", ".join([author.text for author in entry.findall("{http://www.w3.org/2005/Atom}author/{http://www.w3.org/2005/Atom}name")])
        published = entry.find("{http://www.w3.org/2005/Atom}published").text
        summary = entry.find("{http://www.w3.org/2005/Atom}summary").text
        publications.append({
            "title": title,
            "authors": authors,
            "published": published,
            "summary": summary
        })
    return publications


# Example usage
query = "artificial intelligence"
arxiv_publications = search_publications_arxiv(query)

# Print the results
for pub in arxiv_publications:
    print(f"Title: {pub['title']}\nAuthors: {pub['authors']}\nPublished: {pub['published']}\nSummary: {pub['summary']}\n")

Title: ACING: Actor-Critic for Instruction Learning in Black-Box Large Language
  Models
Authors: Salma Kharrat, Fares Fourati, Marco Canini
Published: 2024-11-19T18:58:03Z
Summary:   The effectiveness of Large Language Models (LLMs) in solving tasks vastly
depends on the quality of the instructions, which often require fine-tuning
through extensive human effort. This highlights the need for automated
instruction optimization; however, this optimization is particularly
challenging when dealing with black-box LLMs, where model parameters and
gradients remain inaccessible. We propose ACING, a task-specific prompt
optimization approach framed as a stateless continuous-action Reinforcement
Learning (RL) problem, known as the continuum bandit setting. ACING leverages
an actor-critic-based method to optimize prompts, learning from
non-differentiable reward signals. We validate ACING by optimizing prompts for
ChatGPT on 30 instruction-based tasks. ACING consistently outperforms baseline
metho

**Improvement of query:**

Let's look for our request 'artificial intelligence' in pubmed.

In [ ]:
def search_publications_pubmed(query, max_results=5):
    """
    Searches PubMed for publications based on a query.

    Args:
      query: The search query string.
      max_results: The maximum number of results to retrieve.

    Returns:
      A list of dictionaries, where each dictionary represents a publication
      with "title," "authors," and "url" keys.
    """
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": query,
        "retmax": max_results,
        "retmode": "json"
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()

    data = response.json()
    id_list = data["esearchresult"]["idlist"]

    publications = []
    if id_list:
        base_url_summary = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
        params_summary = {
            "db": "pubmed",
            "id": ",".join(id_list),
            "retmode": "json"
        }
        response_summary = requests.get(base_url_summary, params=params_summary)
        response_summary.raise_for_status()
        summary_data = response_summary.json()
        for pubmed_id, details in summary_data['result'].items():
            if pubmed_id != 'uids':
               publications.append({
                  "title": details.get("title", "No title"),
                  "authors": details.get("authors", "No authors"),
                  "url": f"https://pubmed.ncbi.nlm.nih.gov/{pubmed_id}/"
               })
    return publications

# Example usage:
query = "artificial intelligence"
pubmed_publications = search_publications_pubmed(query)

# Print results
for pub in pubmed_publications:
    print(f"Title: {pub['title']}\nAuthors: {pub['authors']}\nURL: {pub['url']}\n")

Title: Creating an Empirical Dermatology Dataset Through Crowdsourcing With Web Search Advertisements.
Authors: [{'name': 'Ward A', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Li J', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Wang J', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Lakshminarasimhan S', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Carrick A', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Campana B', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Hartford J', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Sreenivasaiah PK', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Tiyasirisokchai T', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Virmani S', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Wong R', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Matias Y', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Corrado GS', 'authtype': 'Author', 'clusterid': ''}, {'name': 'Webster DR', 'authtype': 'Author', 'clusterid': ''}, {'name



### Завдання 4: Створення агента-помічника для вирішення бізнес-задач

Створіть агента, який допомагає вирішувати задачі бізнес-аналітики. Агент має допомогти користувачу створити прогноз по продажам на наступний рік враховуючи рівень інфляції і погодні умови. Агент має вміти використовувати Python і ходити в інтернет аби отримати актуальні дані.

**Кроки:**
1. Налаштуйте агента, який працюватиме з аналітичними даними, заданими текстом. Користувач пише

```
Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т. Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації.
```

2. Створіть запит до агента, що містить чітке завдання – видати результат бізнес аналізу або написати, що він не може цього зробити і запит користувача (просто може бути все одним повідомлленням).

3. Запустіть агента і проаналізуйте результати. Що можна покращити?


In [66]:
my_agent = '/content/drive/MyDrive/Colab Notebooks/data/my_agent.json'

# Read creds.json
with open(my_agent, 'r') as f:
    my_agent = json.load(f)

weather_api_key = my_agent["weather_api_key"]
inflation_api_key = my_agent["inflation_api_key"]

In [83]:
# Define city
city = "Sao Paulo"

# Construct the WeatherAPI URL
url = f"http://api.weatherapi.com/v1/current.json?key={weather_api_key}&q={city}&aqi=no"

# Send the request
response = requests.get(url)

# Check the response status
if response.status_code == 200:
    weather_data = response.json()
    print(weather_data)  # Debugging output

    # Extract weather data
    temperature = weather_data.get('current', {}).get('temp_c', 'No data')
    precipitation = weather_data.get('current', {}).get('precip_mm', 0)
    weather_description = weather_data.get('current', {}).get('condition', {}).get('text', 'No data')

    # Format and print the weather info
    weather_info = f"Temperature in {city}: {temperature}°C, precipitation: {precipitation} mm, condition: {weather_description}."
    print(weather_info)
else:
    print(f"Request error: {response.status_code} - {response.text}")


{'location': {'name': 'Sao Paulo', 'region': 'Sao Paulo', 'country': 'Brazil', 'lat': -23.5333, 'lon': -46.6167, 'tz_id': 'America/Sao_Paulo', 'localtime_epoch': 1732147302, 'localtime': '2024-11-20 21:01'}, 'current': {'last_updated_epoch': 1732147200, 'last_updated': '2024-11-20 21:00', 'temp_c': 21.2, 'temp_f': 70.2, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 8.9, 'wind_kph': 14.4, 'wind_degree': 159, 'wind_dir': 'SSE', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 88, 'cloud': 75, 'feelslike_c': 21.2, 'feelslike_f': 70.2, 'windchill_c': 20.8, 'windchill_f': 69.4, 'heatindex_c': 20.8, 'heatindex_f': 69.4, 'dewpoint_c': 17.5, 'dewpoint_f': 63.5, 'vis_km': 6.0, 'vis_miles': 3.0, 'uv': 0.0, 'gust_mph': 10.3, 'gust_kph': 16.6}}
Temperature in Sao Paulo: 21.2°C, precipitation: 0.0 mm, condition: Partly cloudy.


In [85]:
# Define the country for which you want inflation data
country = "Brazil"  # Replace with your desired country

# Construct the API request URL
url_inflation = f"https://api.api-ninjas.com/v1/inflation?country={country}"

# Set the headers with the API key
headers = {
    'X-Api-Key': inflation_api_key
}

# Execute the request
response_inflation = requests.get(url_inflation, headers=headers)

# Check the response status
if response_inflation.status_code == 200:
    inflation_data = response_inflation.json()
    if inflation_data:
        inflation_rate = inflation_data[0]['yearly_rate_pct']
        print(f"The current inflation rate in {country} is {inflation_rate}%.")
    else:
        print(f"No inflation data available for {country}.")
else:
    print(f"Error: {response_inflation.status_code} - {response_inflation.text}")


Error: 400 - {"error": "The requested country inflation data is for premium subscribers only."}


**Nota bene:**

Unfortunately, the API I used provides data for premium subscribers only. Let's try to bypass with rapidapi.

In [92]:
import http.client

conn = http.client.HTTPSConnection("inflation-by-api-ninjas.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "063dfdbe76mshe5b144c25dfc7fep11f051jsnd8ecd467b712",
    'x-rapidapi-host': "inflation-by-api-ninjas.p.rapidapi.com"
}

conn.request("GET", "/v1/inflation", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

[{"country": "Austria", "type": "CPI", "period": "Oct 2024", "monthly_rate_pct": 0.32, "yearly_rate_pct": 1.81}, {"country": "Belgium", "type": "CPI", "period": "Oct 2024", "monthly_rate_pct": 0.48, "yearly_rate_pct": 3.2}, {"country": "Canada", "type": "CPI", "period": "Oct 2024", "monthly_rate_pct": 0.43, "yearly_rate_pct": 2.02}, {"country": "Chile", "type": "CPI", "period": "Oct 2024", "monthly_rate_pct": 0.98, "yearly_rate_pct": 4.72}, {"country": "Estonia", "type": "CPI", "period": "Oct 2024", "monthly_rate_pct": 0.6, "yearly_rate_pct": 4.08}, {"country": "Germany", "type": "CPI", "period": "Oct 2024", "monthly_rate_pct": 0.42, "yearly_rate_pct": 2.04}, {"country": "Hungary", "type": "CPI", "period": "Oct 2024", "monthly_rate_pct": 0.09, "yearly_rate_pct": 3.17}, {"country": "Iceland", "type": "CPI", "period": "Oct 2024", "monthly_rate_pct": 0.29, "yearly_rate_pct": 5.07}, {"country": "Ireland", "type": "CPI", "period": "Oct 2024", "monthly_rate_pct": 0.3, "yearly_rate_pct": 0.7}

In [98]:
llm = OpenAI(api_key=api_key, temperature=0.3)

prompt = """
Define inflation in Brazil during 2024. Give montly rate for November and yearly rate for 2024. Limit your response to 50 words.
"""

inflation_rate = llm(prompt)
inflation_rate

'\nInflation in Brazil during 2024 refers to the overall increase in prices of goods and services in the country. In November 2024, the monthly inflation rate was 0.5%, while the yearly inflation rate for 2024 was 6.2%. This means that the cost of living in Brazil increased by 6.2% compared to the previous year.'

In [102]:
# Prompt for forecast
prompt_template = PromptTemplate(
    input_variables=["weather", "inflation_rate"],  # Include inflation_rate in input_variables
    template="""
    Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т.
    Враховуючи погодні умови в Бразилії: {weather}, та світовий попит на апельсини з урахуванням економічної ситуації: {inflation_rate} ,
    Зробіть прогноз, скільки тонн апельсинів ми зможемо експортувати у 2025 році.
    """
)

# Form the final prompt
final_prompt = prompt_template.format(
    weather=weather_info,
    inflation_rate=inflation_rate,  # Use inflation_rate instead of inflation
)

# Response from the model with word limit
response = llm(final_prompt, max_tokens=350)  # Assuming ~1.4 tokens per word

# Result
print(f"Прогноз експорту на 2025 год:\n{response}")

Прогноз експорту на 2025 год:

За умови, що погодні умови в Бразилії залишаться стабільними, а світовий попит на апельсини не зменшиться, можна припустити, що експорт апельсинів з Бразилії буде продовжуватися і у 2025 році. Однак, з урахуванням економічної ситуації в країні, можна очікувати зростання цін на апельсини, що може вплинути на обсяг експорту.

З огляду на те, що в 2024 році було експортовано 220 тонн апельсинів, можна припустити, що у 2025 році цей обсяг може збільшитися на 5-10%. Таким чином, можна прогнозувати, що у 2025 році ми зможемо експортувати від 231 до 242 тонн апельсинів з Бразилії.


**Improvement suggestions:**

1. Use Long-term Weather Forecasts: Implement other APIs to obtain long-term weather forecasts, helping to better predict future weather conditions in Brazil that might impact the orange harvest.
2. Detailed Economic Demand Forecasts: Utilize more comprehensive APIs for economic demand forecasts instead of relying only on recent news. This would include analyzing detailed economic indicators that impact global demand for oranges.
3. Analyze Global Production Trends: Incorporate data on global trends in fruit production to predict changes in demand, provided you have access to historical data.
4. Machine Learning Model Training: Train a machine learning model using historical data on exports, weather, and demand. This model could provide more accurate forecasts for 2025.
5. Add Visualization: Enhance the presentation of forecasts with visualizations using libraries like matplotlib or plotly. This will help in understanding the dynamics of demand and weather conditions through clear, interactive graphs.
6. Seasonal Weather Patterns: Include seasonal weather pattern analysis to better predict weather-related impacts on orange production.
7. Economic Indicators: Integrate other economic indicators such as GDP growth, consumer spending, and trade policies to improve demand forecasts.
8. Supply Chain Analysis: Consider the effects of supply chain disruptions and logistics issues on orange exports.
9. Competitor Analysis: Analyze export data from competing countries to understand their impact on global orange supply and demand.
10. Consumer Trends: Monitor changes in consumer preferences and health trends that might affect orange consumption.
11. Policy Changes: Stay updated on agricultural policies and trade agreements that might influence export regulations and tariffs.
12. limate Change Impact: Model the long-term impacts of climate change on orange production in Brazil.
13. Market Prices: Include analysis of current and projected market prices for oranges and related fruits.
14. Weather Event Simulation: Simulate the impact of extreme weather events like droughts or floods on the orange harvest.
15. Real-time Data Integration: Continuously integrate real-time data feeds to update forecasts dynamically.
